找出了比较好的学习率之后，我们再尝试调整正则化因子

正则化因子的初始值是7e-5，现在尝试增大正则化因子

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 1e-3*l2_loss                                             #将正则化因子调整为1e-3
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.1                                                                      #调整学习率为0.1
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 100, entropy los

事实上，最后的结果相差不多，调整后稍微差一些

调小正则参数为1e-4

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 1e-4*l2_loss                                             #将正则化因子调整为1e-3
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.1                                                                      #调整学习率为0.1
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.404953, l2_loss: 38214.531250, total loss: 4.226406
0.89
step 200, entropy loss: 0.193385, l2_loss: 38140.164062, total loss: 4.007401
0.95
step 300, entropy loss: 0.092889, l2_loss: 38064.906250, total loss: 3.899379
0.98
step 400, entropy loss: 0.228373, l2_loss: 37989.882812, total loss: 4.027361
0.98
step 500, entropy loss: 0.155947, l2_loss: 37914.683594, total loss: 3.947415
0.96
step 600, entropy loss: 0.050012, l2_loss: 37839.832031, total loss: 3.833996
0.97
step 700, entropy loss: 0.068647, l2_loss: 37764.898438, total loss: 3.845137
1.0
step 800, entropy loss: 0.179853, l2_loss: 37690.062500, total loss: 3.948859
0.98
step 900, entropy loss: 0.137887, l2_loss: 37615.382812

可以发现，正则化因子对结果的影响不是特别大